In [88]:
import pandas as pd
import chardet

caminho_arquivo = "/home/juniorsilvadev/engenharia-dados/engenharia-dados-com-python-e-banco-de-dados/PostgreSQL/Arquivos JSON/V_OCORRENCIA_AMPLA.json"

with open(caminho_arquivo, "rb") as f:
    raw_data = f.read(10000)
    result = chardet.detect(raw_data)
    encoding_detectado = result['encoding']

df = pd.read_json(caminho_arquivo, encoding=encoding_detectado)

In [89]:
df = pd.read_json(caminho_arquivo, encoding=encoding_detectado)

In [90]:
colunas = ['Numero_da_Ocorrencia', 'Classificacao_da_Ocorrência', 'Data_da_Ocorrencia', 'Municipio', 'UF', 'Regiao', 'Nome_do_Fabricante']
df = df[colunas]

In [91]:
df.rename(columns={'Classificacao_da_Ocorrência': 'Classificacao_da_Ocorrencia'}, inplace=True)
df.columns = df.columns.str.strip().str.lower() # Remover espaços extras e converter os nomes das colunas para minúscula

In [92]:
df['data_da_ocorrencia'] = pd.to_datetime(df['data_da_ocorrencia'])

In [93]:
from datetime import datetime

ano_atual = datetime.now().year
print(ano_atual)

2025


In [94]:
df = df[df['data_da_ocorrencia'].dt.year == 2024]

Conexão avançada com create engine

In [95]:
# pip install sqlalchemy
# Tipos de dados: Integer, String, Date, VARCHAR
# Conexão com PostgreSQL, MySQL, SQLite e muito mais com auto desemprenho

from sqlalchemy import create_engine, Integer, String, Date, VARCHAR, text

dbname   = 'anac_db'
user     = 'postgres'
password = 'root'
host     = 'localhost'
port     = '5432'

conexao_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

try:
    engine = create_engine(conexao_str)
    print("Conexão bem-sucedida ao PostgreSQL!")
except Exception as e:
    print('Error ao conectar com banco de dados:', e)

Conexão bem-sucedida ao PostgreSQL!


In [96]:
nome_tabela = 'anac_sqlalchemy'
ano_2024 = 2024

# Estabelece a conexão
cursor = engine.connect()

# Cria a consulta SQL como texto executável
delete = text(f'DELETE FROM public.{nome_tabela} WHERE EXTRACT(YEAR FROM "data_da_ocorrencia"::DATE) = {ano_2024}')

# Executa a consulta
cursor.execute(delete)
cursor.commit()

# Agora insere os dados do DataFrame na tabela
df.to_sql(nome_tabela, engine, index=False, if_exists='append', dtype={
    'Numero_da_Ocorrencia': Integer,
    'Classificacao_da_Ocorrencia': VARCHAR(50),
    'Data_da_Ocorrencia': Date,
    'Municipio': VARCHAR(50),
    'UF': VARCHAR(30),
    'Regiao': VARCHAR(50),
    'Nome_do_Fabricante': VARCHAR(50)
})

# replace - sobrescreve toda a tabela
# append -  adiciona dados ao fim da tabela

engine.dispose()
cursor.close()